In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from metrics.helper_function import calc_ROI,remove_invalid_values
from metrics import fillrate,temporalNoise,planeTest
from inout import bag_reader
from inout import svo_reader

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

## Configs

In [ ]:
# bag-file 
data_path = "DATA_DIR_PATH" # adjust to the directory to the your data-path
file_name = "FILE_NAME.bag" # adjust to the file name you want to load
distance_mm = 160 # adjust ground truth distance
depth_list,image_list = bag_reader.extract_frames(os.path.join(data_path,file_name),None,2,distance_mm)

In [ ]:
depth = calc_ROI(depth_list[0])

pcd = o3d.geometry.PointCloud()
depth_valid = depth.reshape((-1,3))
depth_valid = remove_invalid_values(depth_valid)
print(depth_valid.shape)
# show the selected/valid points of the chessboard in 3D
pcd.points = o3d.utility.Vector3dVector(depth_valid)
o3d.visualization.draw_geometries([pcd])

## Fillrate

Percentage of “valid” (w/non-zero depth) pixels over ROI 


In [ ]:
z = depth[:,:,-1]
# count invalid values decoded as nan
print("Total values",z.size)
print("Valid values",np.sum(~(np.isnan(z))))

# count invalid vales by min max filtering
print("Point-cloud values:",z.size-np.sum(np.isnan(z)))
print("Point-cloud invalid values:",np.sum((z==0.)))

np.sum(~(np.isnan(z))) / z.size

In [ ]:
fillrate.fillrate(depth_list)

## Temporal noise

Variation in depth per pixel over time (frame-to-frame).


In [ ]:
# count invalid values decoded as nan
depth_list_valid = [np.nan_to_num(depth[:,:,-1]) for depth in depth_list]
# cut the ROI
depth_list_roi = np.array([abs(calc_ROI(depth)) for depth in depth_list_valid])
# remove invalid values
depth_list_roi_valid =  np.ma.masked_equal(depth_list_roi,0)

std_matrix = np.std(depth_list_roi_valid,0)
print(std_matrix.shape)

print("Temporal noise:",np.average(std_matrix))
plt.imshow(std_matrix,cmap='coolwarm')

In [ ]:
temporalNoise.get_tempNoise(depth_list)

## Z-accuracy

Offset of mean/median depth from ground truth. For more detail information see on the [IntelRealsense page](https://github.com/IntelRealSense/librealsense/tree/master/tools/depth-quality#z-accuracy).

* $PlanesOffset_{mm}$ - Distance (signed) from the Fitted to the Ground Truth planes (mm)
* $Di$ - Distance (signed) from a depth vertex to the Fitted Plane (mm)
* $D^{'}_i$ - Z-error: Distance (signed) from the rotated Di coordinate to the Ground Truth Plane (mm)
* $GT$ - Ground Truth distance to the wall (mm)

$$ D^{'}_{mm} = D_i - PlanesOffset_{mm} $$

$$ Z - Accuracy =  100 \times median({ \sum_{1}^n (D^{'}_i - GT) \over GT}) $$

Note that to compare the ground truth distance to the measured depth, the offset between the camera's optical center and the case must be added.

In [ ]:
a, b, c, d = planeTest.plane_fit(depth_valid)
print(f"The plane equation of plane_fit is {a:.2f}x + {b:.2f}y + {c:.2f}z = {d:.2f}")
a, b, c, d = planeTest.plane_from_points(depth_valid)
print(f"The plane equation of fast plane_fit is {a:.2f}x + {b:.2f}y + {c:.2f}z = {d:.2f}")

In [ ]:
planeTest.get_Z_accuracy(depth_list,distance_mm)

## Chessboard-distance

For the chessboard distance check the [chessboard.ipynb](./chessboard.ipynb) notebook.